In [4]:
!pip install transformers

     |████████████████████████████████| 1.9MB 7.9MB/s 
     |████████████████████████████████| 3.2MB 41.8MB/s 
     |████████████████████████████████| 890kB 28.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9b1f65f78696ecce34514a38b824d95cc8d08b2f22f5eac752d137aa81a78534
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [7]:
df_raw = pd.read_csv('cd.csv')
df_raw.head(3)

,id,tweet,label,domain
0,2.263108e+16,@FC_HELP can I return online purchases to a Ho...,0,apparel
1,2.336408e+16,@FC_Help Hi - I'm writing a piece for MSN Him ...,0,apparel
2,2.555041e+16,@FC_Help i need to check my order,0,apparel


In [9]:
df_alt = pd.DataFrame(df_raw[['tweet', 'label']])
df_0 = df_alt.loc[df_alt.label == 0]
df_1 = df_alt.loc[df_alt.label == 1]
print(len(df_0), len(df_1))

2217 1232


In [10]:
df_list = [df_0[:1000], df_1[:1000]]
df_fin = pd.concat(df_list)

In [11]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [14]:
tokenized = df_fin['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [18]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [20]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [24]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [25]:
features = last_hidden_states[0][:,0,:].numpy()


In [26]:
labels = df_fin.label

In [29]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [30]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
lr_clf.score(test_features, test_labels)

0.85